In [5]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi import NewsApiClient
from finvizfinance.screener.overview import Overview
import alpaca_trade_api as tradeapi
import datetime
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
import arch
import math
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


In [8]:
# Read your api key environment variable
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("news_api")

In [10]:
# Create a newsapi client
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key=os.environ["news_api"])

In [11]:
foverview = Overview()

filters_dict = {
    'Performance':'Today +5%',
    'Current Volume':"Over 5M",
    'Country': 'USA'
}
foverview.set_filter(filters_dict=filters_dict)

tickers = foverview.ScreenerView()
#tickers.sort_values(by='Volume', inplace=True, ascending=False)
tickers.set_index('Ticker', inplace=True )
tickers.head(100)

,Company,Sector,Industry,Country,Market Cap,P/E,Price,Change,Volume
Ticker,,,,,,,,,
AA,Alcoa Corporation,Basic Materials,Aluminum,USA,4.250000e+09,None,23.13,0.0952,7212741.0
AAL,American Airlines Group Inc.,Industrials,Airlines,USA,1.142000e+10,None,18.68,0.0548,47576082.0
AEZS,Aeterna Zentaris Inc.,Healthcare,Biotechnology,USA,9.223000e+07,None,1.48,0.0571,13560255.0
AHT,"Ashford Hospitality Trust, Inc.",Real Estate,REIT - Hotel & Motel,USA,2.416200e+08,None,3.76,0.0867,15717857.0
AIKI,AIkido Pharma Inc.,Healthcare,Biotechnology,USA,4.598000e+07,None,1.62,0.0519,42328806.0
...,...,...,...,...,...,...,...,...,...
VTGN,"VistaGen Therapeutics, Inc.",Healthcare,Biotechnology,USA,4.250900e+08,None,2.96,0.1935,19624977.0
WATT,Energous Corporation,Technology,Scientific & Technical Instruments,USA,3.146500e+08,None,7.23,0.9435,176508809.0
WKHS,Workhorse Group Inc.,Consumer Cyclical,Auto Manufacturers,USA,4.110000e+09,None,32.96,0.0557,9563070.0


In [12]:
def get_sentiment(company):
    stock_headlines = ( newsapi.get_everything(
    q=company,
    language='en',
    sort_by='relevancy'))

    sentiments = []

    for articles in stock_headlines['articles']:
        try:
            text = articles['content']
            results = analyzer.polarity_scores(text)
            compound = results['compound']
            pos = results['pos']
            neu = results['neu']
            neg = results['neg'] 

            #put data into sentiments
            sentiments.append({
                'text':text,
                'Compound':compound,
                'Positive':pos,
                'Negative':neg,
                'Neutral':neu})

        except AttributeError:
            pass
    fininal_sentiments = pd.DataFrame(sentiments)
    if not fininal_sentiments.empty:
        return(fininal_sentiments["Positive"].mean())
    else:
        return(0)

In [13]:
ticker_sentiment = [] 
for ticker in tickers.index: 
    sentiment = get_sentiment(tickers['Company'][ticker])
    print(sentiment)
    if float(sentiment) >= 0.1:
        ticker_sentiment.append(ticker)
        print(f"{ticker} meets our sentiment requirement")
    else:
        print(f"{ticker} does not meet our sentiment requirement")

NewsAPIException: {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

In [14]:
### Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2",
    base_url = "https://paper-api.alpaca.markets"
)
account = api.get_account()
print(account.status)

ACTIVE


In [15]:
#Set time frame for reporting stocks
today = datetime.date.today()
tomorrow = today + datetime.timedelta(days = 2) 
start_time = tomorrow.strftime("%b %d %Y") + " 10:00AM"
next_day = today + datetime.timedelta(days = 1) 
end_time = next_day.strftime("%b %d %Y") + " 1:00PM"

date_from = datetime.datetime.strptime(
    start_time, '%b %d %Y %I:%M%p')
date_to = datetime.datetime.strptime(
    end_time, '%b %d %Y %I:%M%p')

In [ ]:
# Set timeframe to '1D'
timeframe = "1D"

#Set timeframe for last 2 years
start_date = pd.Timestamp("2018-05-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp(next_day, tz="America/New_York").isoformat()

#Get ticker data
df_ticker = api.get_barset(
    ticker_sentiment,
    timeframe,
    start=start_date,
    end=end_date,
).df

df_ticker.dropna()
df_ticker.tail()


In [ ]:
df_closing_prices = pd.DataFrame()
for ticker in ticker_sentiment: 
    df_closing_prices[ticker] = df_ticker[ticker]["close"]

In [ ]:
df_closing_prices.dropna(inplace=True)
df_closing_prices.tail()

In [ ]:
def hpfilter(ticker):
    ts_noise, ts_trend = sm.tsa.filters.hpfilter(df_ticker[ticker][['close']])

    #create different dataframes for each composition
    ticker_decomp = df_ticker[ticker][['close']].copy()
    ticker_decomp['noise'] = ts_noise
    ticker_decomp['trend'] = ts_trend
    return ts_noise.plot(title='Noise')

In [ ]:
for ticker in ticker_sentiment: 
    print(hpfilter((ticker)))

In [ ]:
#Plot ACF & PACF
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf 
plot_acf(df.df_closing_prices, zero=False, lags=35) 

In [ ]:
plot_pacf(df.df_closing_prices, zero=False, lags=35) 

But you need to be careful to not over-difference the series. Because, an over differenced series may still be stationary, which in turn will affect the model parameters.

So how to determine the right order of differencing?

The right order of differencing is the minimum differencing required to get a near-stationary series which roams around a defined mean and the ACF plot reaches to zero fairly quick.

If the autocorrelations are positive for many number of lags (10 or more), then the series needs further differencing. On the other hand, if the lag 1 autocorrelation itself is too negative, then the series is probably over-differenced.

In the event, you can’t really decide between two orders of differencing, then go with the order that gives the least standard deviation in the differenced series.


In [ ]:
price_forcasting = pd.DataFrame()
dfs=[]
for ticker in df_closing_prices: 
    model = ARIMA(df_closing_prices[ticker], order=(1,1,1))
    results = model.fit()
    df = pd.DataFrame(results.forecast(steps=15)[0], columns=[ticker])
    dfs.append(df)

final_df = pd.concat(dfs, axis=1)
final_df.head(20)

In [ ]:
#Evaluate Arima Model using metrics
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    acf1 = acf(fc-test)[1]                      # ACF1
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'acf1':acf1, 
            'corr':corr, 'minmax':minmax})

forecast_accuracy(fc, test.values)

In [16]:
#plot Arima Model 
model.plot_diagnostics(figsize=(7,5))
plt.show()

NameError: name 'model' is not defined

Top left: The residual errors seem to fluctuate around a mean of zero and have a uniform variance.

Top Right: The density plot suggest normal distribution with mean zero.

Bottom left: All the dots should fall perfectly in line with the red line. Any significant deviations would imply the distribution is skewed.

Bottom Right: The Correlogram, aka, ACF plot shows the residual errors are not autocorrelated. Any autocorrelation would imply that there is some pattern in the residual errors which are not explained in the model. So you will need to look for more X’s (predictors) to the model.

In [ ]:
df_closing_list = df_closing_prices.iloc[[-1]].values.tolist()
final_df_list = final_df.iloc[[9]].values.tolist()


In [ ]:
base_price = df_closing_list[0]
enter_price = final_df_list[0]
print(enter_price,base_price)

In [ ]:
investment = 2500

stock_capital = round(investment / len(stock_buys),2)
print(stock_capital)

In [ ]:

stock_buys=[]
number_of_shares = {}
num = 0 
for ticker in df_closing_prices: 
    value = round((enter_price[num] - base_price[num])/ base_price[num] * 100.0,2)
    if value > 7:
        stock_buys.append(ticker)
        number_of_shares[ticker] = math.floor(stock_capital/enter_price[num])
        api.submit_order(
            symbol=ticker,
            side='buy',
            type='limit',
            limit_price=enter_price[num],
            qty=number_of_shares[ticker],
            time_in_force='day',
            take_profit=dict(
                limit_price=enter_price[num]*1.50,
            ),
            stop_loss=dict(
                stop_price=enter_price[num]/1.20,
            )
        )
    num += 1


In [ ]:
print(stock_buys)

In [ ]:
print(number_of_shares)

In [ ]:
print(api.get_clock())